In [1]:
import numpy as np
import faiss

下面我们构建了shape为[100000,64]的训练数据xb，和shape为[10000,64]的查询数据xq。
然后创建索引(Index)。faiss创建索引对向量预处理，提高查询效率。
faiss提供多种索引方法，这里选择最简单的暴力检索L2距离的索引：IndexFlatL2。
创建索引时必须指定向量的维度d。大部分索引需要训练的步骤。IndexFlatL2跳过这一步。
当索引创建好并训练(如果需要)之后，我们就可以执行add和search方法了。add方法一般添加训练时的样本，search就是寻找相似相似向量了。
一些索引可以保存整型的ID，每个向量可以指定一个ID，当查询相似向量时，会返回相似向量的ID及相似度(或距离)。如果不指定，将按照添加的顺序从0开始累加。其中IndexFlatL2不支持指定ID。

In [2]:
d = 64                           # dimension
nb = 100000                      # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.

xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000

In [3]:
print(xq.shape)
xq

(10000, 64)


array([[ 0.81432974,  0.7409969 ,  0.8915324 , ...,  0.72459674,
         0.893881  ,  0.6574571 ],
       [ 0.5844774 ,  0.797842  ,  0.74140453, ...,  0.6768835 ,
         0.05907924,  0.6396156 ],
       [ 0.75040764,  0.02659794,  0.5495097 , ...,  0.69562465,
         0.16268532,  0.76653737],
       ...,
       [10.96773   ,  0.05037309,  0.7342035 , ...,  0.89510185,
         0.6490696 ,  0.86151606],
       [10.831193  ,  0.70606154,  0.1922274 , ...,  0.8026039 ,
         0.6854174 ,  0.60209423],
       [10.078484  ,  0.39106598,  0.01359335, ...,  0.63193923,
         0.12561724,  0.78384215]], dtype=float32)

In [5]:
print(xb.shape)

(100000, 64)


In [6]:
index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)
index.add(xb)                  # add vectors to the index
print(index.ntotal)

True
100000


In [7]:
k = 4                          # we want to see 4 nearest neighbors
D, I = index.search(xb[:5], k) # sanity check
print(I)
print(D)
D, I = index.search(xq, k)     # actual search
print(I[:5])                   # neighbors of the 5 first queries
print(I[-5:]) 

[[  0 393 363  78]
 [  1 555 277 364]
 [  2 304 101  13]
 [  3 173  18 182]
 [  4 288 370 531]]
[[0.        7.175174  7.2076287 7.251163 ]
 [0.        6.323565  6.684582  6.799944 ]
 [0.        5.7964087 6.3917365 7.2815127]
 [0.        7.277905  7.5279875 7.6628447]
 [0.        6.763804  7.295122  7.368814 ]]
[[ 381  207  210  477]
 [ 526  911  142   72]
 [ 838  527 1290  425]
 [ 196  184  164  359]
 [ 526  377  120  425]]
[[ 9900 10500  9309  9831]
 [11055 10895 10812 11321]
 [11353 11103 10164  9787]
 [10571 10664 10632  9638]
 [ 9628  9554 10036  9582]]
